In [1]:
import subproptools
from subproptools import qtaimExtract as qt
import pandas as pd
import numpy as np
import math



# Accessing documentation


All user facing functions are thoroughly documented, doc strings can be accessed as below

In [2]:
help(qt.extract_sub_props)

Help on function extract_sub_props in module subproptools.qtaimExtract:

extract_sub_props(data: list[str], subAtoms: list[int], sumFileNoExt: str, groupProps=True, bcpId=[[1, 2]], lapRhoCpAtoms=[])
    returns a dictionary of all group properties - bcp, group, atomic, and vscc
    Args:
        data: list[str] - lines of a .sum file
        subAtoms - indices of atoms in the molecule comprising group - starts at 1
        groupProps - boolean - do you want to compute group properties
        bcpId - list of 2 length lists. Pass empty list if no bcp properties wanted. 2 length lists are indices of atoms that you want BCPs for
        lapRhoCpAtoms = list of atom indices that you want to find laprhoCPs for. Defaults to empty
    vol surface is 0.001 au isodensity
    
    Returns:
        nested dictionaries
        {'Group': dictionary of group properties,
        'Atomic': 
            {'C1':dictionary of properties for C1, 
             'H2':dictionary of properties for H2,...},
    

# User facing functions

The following are the user facing functions that you are intended to call. Examples of each of these will be provided.

* extract_sub_props - gets substituent properties for a batch of sum files found defined by input file
* get_atomic_props - gets atomic properties for all atoms in file
* get_bcp_properties - gets BCP properties for selected bonds
* get_cc_props - gets VSCC properties
* get_sub_di - gets the delocalization index between one substituent and the remainder of the molecule
* get_sub_props - gets substituent properties for one substituent and one sum file
* get_xyz - gets atom list and xyz coordinates in sum file

# get_atomic_props

The simplest function, will extract a variety of properties for a sum file. Note that the .sum file is passed as the variable 'data' which includes a list of the lines of the file. I don't anticipate this being called often, but it has functionality that may be important in some cases, if you're looking for specific atoms in a small study.

This returns a dictionary of one element lists. These are lists as extract_sub_prop will later take these types of dictionaries to create data frames, which require lists.

Usage as below



In [3]:
#create the data object
sumFile = open('SubH_POCH3H_wb97xd_aug-cc-pvtz_reor.sum',"r") #open file, read lines, close file
data = sumFile.readlines()
sumFile.close()
atomic_props = qt.get_atomic_props(data)
#The keys of the outer dictionary are the atoms
list(atomic_props.keys())

['Atom', 'Charge', 'X', 'Y', 'Z']
['Atom', 'q(A)', 'L(A)', 'K(A)', 'K_Scaled(A)', '|Mu_Intra(A)|']
['Atom', 'Mu_Intra_X(A)', 'Mu_Bond_X(A)', 'Mu_X(A)']
['Atom', 'Mu_Intra_Y(A)', 'Mu_Bond_Y(A)', 'Mu_Y(A)']
['Atom', 'Mu_Intra_Z(A)', 'Mu_Bond_Z(A)', 'Mu_Z(A)']
['Atom', '|Mu_Intra(A)|', '|Mu_Bond(A)|', '|Mu(A)|']
['Atom', 'Q_XX(A)', 'Q_XY(A)', 'Q_XZ(A)', 'Q_YY(A)', 'Q_YZ(A)', 'Q_ZZ(A)']
['Atom', 'R-2(A)', 'R-1(A)', 'R0(A)', 'R+1(A)', 'R+2(A)']
['Atom', 'Area(A)', 'Vol(A)', 'N(Vol(A))', 'N(Vol(A))/Vol(A)', '%N(Vol(A))']


['P1', 'H2', 'H3', 'O4', 'C5', 'H6', 'H7', 'H8']

In [4]:
# Each of the above keys is itself a dictionary, each with the following keys. Note Mu_Intra corresponds to mu^p in KLG's papers, and Mu_Bond corresponds to mu^c
list(atomic_props['P1'].keys())

['xyz',
 'q',
 'K',
 'K_Scaled',
 'Mu_Intra_X',
 'Mu_Intra_Y',
 'Mu_Intra_Z',
 'Mu_Bond_X',
 'Mu_Bond_Y',
 'Mu_Bond_Z',
 'Mu_X',
 'Mu_Y',
 'Mu_Z',
 '|Mu_Intra|',
 '|Mu_Bond|',
 '|Mu|',
 'Q_XX',
 'Q_XY',
 'Q_XZ',
 'Q_YY',
 'Q_YZ',
 'Q_ZZ',
 'R+2',
 'R+1',
 'Vol',
 'quadContrib']

In [5]:
#quad contrib specifically is also a dictionary. This contains the atomic contribution to the molecular quadrupole moment, as opposed to Q_XX which is the atomic quadrupole
list(atomic_props['P1']['quadContrib'].keys())

['Q_xx', 'Q_xy', 'Q_xz', 'Q_yy', 'Q_yz', 'Q_zz']

In [6]:
# Example - access x-component of dipole for atom H2, without the [0] at the end, this would be a list, as described above:
atomic_props['H2']['Mu_X'][0]

0.44284903526

# get_bcp_properties

get_bcp_properties takes the data object created earlier and a list of labels of the atoms for which the bcp should be found. See below. It returns a dictionary, see below for keys.

This is also a dictionary of single element lists

In [13]:
qt.get_bcp_properties(data,['P1','H2'])

{'xyz': array([-1.28673557e+00, -1.84401138e-05,  4.95375845e-03]),
 'rho': [0.17523377789],
 'lambda1': [-0.30195221336],
 'lambda2': [-0.28264812538],
 'lambda3': [0.44212073332],
 'DelSqRho': [-0.14247960542],
 'Ellipticity': [0.068297243967],
 'V': [-0.33081070884],
 'G': [0.14759540374],
 'H': [-0.18321530510000003]}

# get_cc_props and identify_vscc

get_cc_props Takes the file name and the atom label you want to find ccs for and returns a dicionary, each key in the outermost being a numerical label for the cc, and the subdictionary contains the properties for the cc

You can narrow this down further to valence shell charge concentrations using identify_vscc, which also requires the atomic propert dictionary

In [16]:
o_ccs = qt.get_cc_props('SubH_POCH3H_wb97xd_aug-cc-pvtz_reor', 'O4')
o_vsccs = qt.identify_vscc(o_ccs,atomic_props)


AttributeError: module 'subproptools.qtaimExtract' has no attribute 'identify_vscc'

In [20]:
o_ccs[2]['rho']

0.30803830854

# extract_sub_props



In [11]:
pohch3_props = qt.extract_sub_props(data=data, subAtoms=[1,3,4,5,6,7,8],sumFileNoExt='SubH_POCH3H_wb97xd_aug-cc-pvtz_reor',bcpId=[[1,2],[1,3]],lapRhoCpAtoms=[4])


['Atom', 'q(A)', 'L(A)', 'K(A)', 'K_Scaled(A)', '|Mu_Intra(A)|']
['Atom', 'Charge', 'X', 'Y', 'Z']
['Atom', 'q(A)', 'L(A)', 'K(A)', 'K_Scaled(A)', '|Mu_Intra(A)|']
['Atom', 'Mu_Intra_X(A)', 'Mu_Bond_X(A)', 'Mu_X(A)']
['Atom', 'Mu_Intra_Y(A)', 'Mu_Bond_Y(A)', 'Mu_Y(A)']
['Atom', 'Mu_Intra_Z(A)', 'Mu_Bond_Z(A)', 'Mu_Z(A)']
['Atom', '|Mu_Intra(A)|', '|Mu_Bond(A)|', '|Mu(A)|']
['Atom', 'Q_XX(A)', 'Q_XY(A)', 'Q_XZ(A)', 'Q_YY(A)', 'Q_YZ(A)', 'Q_ZZ(A)']
['Atom', 'R-2(A)', 'R-1(A)', 'R0(A)', 'R+1(A)', 'R+2(A)']
['Atom', 'Area(A)', 'Vol(A)', 'N(Vol(A))', 'N(Vol(A))/Vol(A)', '%N(Vol(A))']
didnt find first


{'Group': {'q': [0.5701417182699997],
  'K': [455.16423998141994],
  'K_Scaled': [-456.90069639232996],
  'Mu_Intra_X': [0.785635644731],
  'Mu_Intra_Y': [-0.5157322052219999],
  'Mu_Intra_Z': [-0.07011878188800007],
  'Mu_Bond_X': [-1.7664287328238002],
  'Mu_Bond_Y': [1.1778310755774999],
  'Mu_Bond_Z': [1.5100112729350998],
  'Mu_X': [-0.9807930880522],
  'Mu_Y': [0.6620988703638001],
  'Mu_Z': [1.4398924910238],
  'Q_xx': [2.776188704779253],
  'Q_xy': [1.556861915547782],
  'Q_xz': [2.8256223793180855],
  'Q_yy': [-0.8732973727444326],
  'Q_yz': [-3.5093604596778576],
  'Q_zz': [-1.9028913320348213],
  'Vol': [499.60210335299996],
  '|Mu_Intra|': [0.9424010384907786],
  '|Mu_Bond|': [1.8637651089702034]},
 'Atomic': {'P1': {'xyz': array([0., 0., 0.]),
   'q': [3.194228312],
   'K': [338.99681273],
   'K_Scaled': [-340.29008917],
   'Mu_Intra_X': [0.12854229982],
   'Mu_Intra_Y': [-0.15466461987],
   'Mu_Intra_Z': [-0.19334899678],
   'Mu_Bond_X': [-0.30289461204],
   'Mu_Bond_Y': 

In [8]:
type(data)

list